<a href="https://colab.research.google.com/github/SergeyDLevashov/Impulse1/blob/main/TLE_Impuse_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Установка библиотек
!pip install skyfield folium

from skyfield.api import Topos, load, EarthSatellite
import folium
from datetime import datetime

# для определения времени создания TLE
from datetime import datetime, timedelta

# для работы с гаверсинусами :-()
import math

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.9/453.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.3/232.3 kB 5.7 MB/s eta 0:00:00


In [5]:
# Список TLE для нашего спутника с разными эпохами
# получить старые TLE можно на https://www.space-track.org/#/gp
# HISTORICAL ELSET SEARCH
# Entries : 2023-091R
#

tle_txt = """1 57181U 23091R   24064.10585441  .00005510  00000-0  39132-3 0  9993
2 57181  97.6274 117.0436 0019464 119.7568 240.5597 15.04966534 37658
1 57181U 23091R   24064.43829840  .00004543  00000-0  32345-3 0  9997
2 57181  97.6274 117.3719 0019477 118.7401 241.5786 15.04965856 37707
1 57181U 23091R   24064.57127542  .00004060  00000-0  28957-3 0  9990
2 57181  97.6274 117.5033 0019472 118.3091 242.0102 15.04965853 37721
1 57181U 23091R   24064.83722895  .00004149  00000-0  29580-3 0  9992
2 57181  97.6274 117.7660 0019462 117.4763 242.8445 15.04968131 37768
1 57181U 23091R   24064.90371707  .00004029  00000-0  28739-3 0  9994
2 57181  97.6274 117.8317 0019461 117.2796 243.0416 15.04968698 37771"""

In [7]:

# Указать время наблюдения
ts = load.timescale()
# observer_time = datetime.utcnow()
t = ts.utc(2024, 3, 4, 20-3, 7+6, 10) # Время начала PZ0028, 2 пролёт, -3 часа для UTC, +6 минуты к середине пролета

# Указать место наблюдателя (точка прицеливания)
observer_location = Topos(latitude_degrees=55.7558, longitude_degrees=37.6173)


def haversine(lat, lon, latold, lonold, altitude=500):
    R = 6371 + altitude  # ~~~ Радиус Земли в км с учетом высоты
    phi1 = math.radians(lat)
    phi2 = math.radians(latold)
    delta_phi = math.radians(latold - lat)
    delta_lambda = math.radians(lonold - lon)

    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c  # Расстояние в километрах
    return distance * 1000  # Преобразование в метры

m = folium.Map(location=[observer_location.latitude.degrees, observer_location.longitude.degrees], zoom_start=3)
#ts = load.timescale()
#t = ts.utc(observer_time.year, observer_time.month, observer_time.day, observer_time.hour, observer_time.minute, observer_time.second)
tle_data = tle_txt.strip().split("\n")
distance_meters=0

for i in range(0, len(tle_data), 2):
    satellite = EarthSatellite(tle_data[i], tle_data[i+1], "aaa")
    print ("")
    print (satellite) # параметры спутника и эпоха TLE
    print (tle_data[i])
    print (tle_data[i+1])

    # положение спутника в топоцентрической системе
    difference = satellite - observer_location
    topocentric = difference.at(t)
    alt, az, distance = topocentric.altaz()
    print (t, t.utc_strftime()," Distance:" ,distance.km," Alt:",alt.degrees," Az:",az.degrees)


    # координаты проекции спутника
    subpoint = satellite.at(t).subpoint()
    lat, lon = subpoint.latitude.degrees, subpoint.longitude.degrees

    if (i>0):
      distance_meters = haversine(lat, lon, latold, lonold)
      print(f"Отличие от предыдущего положения: {distance_meters:.2f} метров")
    latold=lat
    lonold=lon

    # Трек от спутника длинной в секунду
    t_shifted1 = t + timedelta(minutes=-1)
    subpoint1 = satellite.at(t_shifted1).subpoint()
    lat1, lon1 = subpoint1.latitude.degrees, subpoint1.longitude.degrees
    folium.PolyLine([(lat, lon), (lat1, lon1)], color="grey").add_to(m)

    # Подпись к точке и попап
    folium.Marker([lat,lon],popup=tle_data[i] + " \n" + tle_data[i+1],
      icon=folium.DivIcon(html=f"""<div style="font-family: courier new; color: blue">{str(satellite).replace(" ", "&nbsp;").replace("-","_")}&nbsp;отличие:&nbsp;{distance_meters:.2f}</div>""")).add_to(m)


m



aaa catalog #57181 epoch 2024-03-04 02:32:26 UTC
1 57181U 23091R   24064.10585441  .00005510  00000-0  39132-3 0  9993
2 57181  97.6274 117.0436 0019464 119.7568 240.5597 15.04966534 37658
<Time tt=2460374.2182775927> 2024-03-04 17:13:10 UTC  Distance: 1790.0312822880235  Alt: 10.540980673323558  Az: 355.81495711827426

aaa catalog #57181 epoch 2024-03-04 10:31:09 UTC
1 57181U 23091R   24064.43829840  .00004543  00000-0  32345-3 0  9997
2 57181  97.6274 117.3719 0019477 118.7401 241.5786 15.04965856 37707
<Time tt=2460374.2182775927> 2024-03-04 17:13:10 UTC  Distance: 1788.237543014323  Alt: 10.569034913532885  Az: 355.8337398655587
Отличие от предыдущего положения: 2054.79 метров

aaa catalog #57181 epoch 2024-03-04 13:42:38 UTC
1 57181U 23091R   24064.57127542  .00004060  00000-0  28957-3 0  9990
2 57181  97.6274 117.5033 0019472 118.3091 242.0102 15.04965853 37721
<Time tt=2460374.2182775927> 2024-03-04 17:13:10 UTC  Distance: 1788.055438404013  Alt: 10.572139028433119  Az: 355.835

In [42]:
import math

def haversine(lat1, lon1, lat2, lon2, altitude=500):
    R = 6371 + altitude  # Радиус Земли в км с учетом высоты
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c  # Расстояние в километрах
    return distance * 1000  # Преобразование в метры

# Координаты в градусах
lat = -72.00395294919825
lon = 115.17800145510517
lat_old = -72.0029267447357
lon_old = 115.1764634819688

distance_meters = haversine(lat, lon, lat_old, lon_old)
print(f"Расстояние между точками: {distance_meters:.2f} метров")

Расстояние между точками: 135.62 метров
